In [1]:
import simplesimdb  # A leightweight data creator and database manager
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml # for output files

pd.set_option('display.float_format', lambda x: '%.2e' % x)
# Type make solvers in the repository to have the executable available
database = simplesimdb.Manager( executable='./solvers', directory='data', filetype = 'yaml')

In [2]:
#define conversion function 
def orderToString(x): 
    if np.isnan(x) : return 'n/a'
    return'%.2f'% x

def errorToString(x):
    return '%.2e' % x

In [3]:
def create_solver() :
    return {
        "type" : "CG", # CG, LGMRES, BICGSTABl, Multigrid
        "eps" : 1e-6,
        "preconditioner" :{
            "type" : "diagonal" # None or diagonal
        },
        "max_iter" : 50000, # maximum number of iterations
        # for CG
        "check-every" : 1, # errror check every .. iterations
        # for LGMRES
        "inner_m" : 30,
        "outer_k" : 3,
        # for BICGSTABL
        "l_input" : 3
    }


In [4]:
def create_inputfile ():
    '''Create an input file for the solvers FELTOR code'''
    return {
        "grid" : {
            "n" : 3,
            "Nx" : 128, 
            "Ny" : 256,
        },
        "solver" : create_solver(),
        "equations":
        {
            "amp" : 0.9, # The closer to 1 the more difficult!   
        },
        "elliptic" :
        {
            "jfactor" : 1,
            "direction" : "forward" # forward or centered

        }    
    }


In [5]:
def make_dataframe():
    '''Make a pandas Dataframe of all simulations in the database
    
    Concatenates all inputs and outputs as serialized dicts
    '''
    inputs  = list()
    outputs =list()
    for data in database.table() :
        inputs.append( data)
        with open( database.outfile( data)) as f:
            output = yaml.full_load(f)
            outputs.append( output)
    dfi = pd.json_normalize(inputs)        
    dfo = pd.json_normalize(outputs)
    return pd.concat([dfi, dfo], axis=1).sort_values( 'grid.Nx', ascending=True)

In [6]:
# Clean out all existing simulations
database.delete_all()
database.directory='data'

In [7]:
string_dflist = []
string_directions = []
string_ns = []
ns = [3,4]
for n in ns :
    for direction in ['forward','backward','centered']:
        inputfile = create_inputfile()
        for (i,eps) in zip([17,34,68,136],[1e-6,1e-7,1e-8,1e-9]):
            inputfile["grid"]["n"] = n
            inputfile["grid"]["Nx"] = i
            inputfile["grid"]["Ny"] = i
            inputfile["solver"]["eps"] = eps
            inputfile["elliptic"]["direction"] = direction
            database.create( inputfile, error = 'display')

        df = make_dataframe()
        dff = df.loc [ (df["grid.n"]==n) & (df['elliptic.direction'] == direction)]
        dff = dff[['grid.n','grid.Nx','grid.Ny',
                  'iter','time','error','error_var_fw', 'error_var_bw', 'error_var_cc']]

        for name in ['error','error_var_fw', 'error_var_bw', 'error_var_cc']:
            dff.insert(dff.columns.get_loc(name)+1,column="order."+name, value=dff.loc[:,name])
            dff["order."+name] = (np.log ( dff["order."+name]/ dff["order."+name].shift(1))/
                np.log( dff["grid.Nx"].shift(1)/dff["grid.Nx"]))
        dff.set_index(['grid.n','grid.Nx','grid.Ny','iter','time'], inplace=True)
        headers = [ "$\phi$",
                    "$(\nabla\phi)^2_{FW}$",
                    "$(\nabla\phi)^2_{BW}$",
                    "$(\nabla\phi)^2_{CC}$"]
        dff.columns=pd.MultiIndex.from_product([headers,["error", "order"] ])
        #We would like the order to be displayed differently.
        #In pandas we must convert the columns to strings to do that.
        string_dff = dff.copy()
        for header in headers :
            string_dff[header, "error"]=string_dff[header, "error"].apply( errorToString)
            string_dff[header, "order"]=string_dff[header, "order"].apply( orderToString)
        string_dflist.append(string_dff)
        string_directions.append(direction)
        string_ns.append(n)

Running simulation ce55d3...edb8.yaml
Running simulation a85843...b013.yaml
Running simulation f2036e...e5da.yaml
Running simulation 0aef15...b3bf.yaml
Running simulation f8107f...d77c.yaml
Running simulation 1692e9...e489.yaml
Running simulation 05a3b7...7fe2.yaml
Running simulation 2aa3eb...76f0.yaml
Running simulation 3601a5...6e15.yaml
Running simulation a31d12...bb49.yaml
Running simulation 562091...c3d8.yaml
Running simulation b4ce5b...830e.yaml
Running simulation 72e3f7...1d78.yaml
Running simulation ec29ae...e4ab.yaml
Running simulation 91ae4f...9197.yaml
Running simulation 6482b5...30a4.yaml
Running simulation 2cd97b...7e2c.yaml
Running simulation d9be8b...625c.yaml
Running simulation 5e8c3f...2b9a.yaml
Running simulation dc7da6...e9d1.yaml
Running simulation 992e20...b529.yaml
Running simulation b737e0...6600.yaml
Running simulation 072554...ae84.yaml
Running simulation cab0b6...519f.yaml


In [8]:
for i in range(0,len(string_dflist)):
    print( f"Direction is {string_directions[i]}, n is {string_ns[i]}")
    display( string_dflist[i])

Direction is forward, n is 3


$\phi$       $(\nabla\phi)^2_{FW}$  \
                                         error order                 error   
grid.n grid.Nx grid.Ny iter time                                             
3      17      17      241  3.15e-02  1.87e-04   n/a              9.65e-04   
       34      34      516  1.33e-01  2.32e-05  3.01              1.29e-04   
       68      68      1075 7.10e-01  2.92e-06  2.99              1.68e-05   
       136     136     2250 7.53e+00  3.67e-07  2.99              2.14e-06   

                                           $(\nabla\phi)^2_{BW}$        \
                                     order                 error order   
grid.n grid.Nx grid.Ny iter time                                         
3      17      17      241  3.15e-02   n/a              1.04e-02   n/a   
       34      34      516  1.33e-01  2.90              2.65e-03  1.97   
       68      68      1075 7.10e-01  2.94              6.71e-04  1.98   
       136     136     2250 7.53e+00  2.97              1.69e-04  1.99   

                                     $(\nabla\phi)^2_{CC}$        
                                                     error order  
grid.n grid.Nx grid.Ny iter time                                  
3      17      17      241  3.15e-02              5.26e-03   n/a  
       34      34      516  1.33e-01              1.33e-03  1.98  
       68      68      1075 7.10e-01              3.36e-04  1.98  
       136     136     2250 7.53e+00              8.45e-05  1.99

Direction is backward, n is 3


$\phi$       $(\nabla\phi)^2_{FW}$  \
                                         error order                 error   
grid.n grid.Nx grid.Ny iter time                                             
3      17      17      241  3.11e-02  1.87e-04   n/a              1.04e-02   
       34      34      514  1.17e-01  2.32e-05  3.01              2.65e-03   
       68      68      1074 7.57e-01  2.92e-06  2.99              6.71e-04   
       136     136     2243 7.75e+00  3.67e-07  2.99              1.69e-04   

                                           $(\nabla\phi)^2_{BW}$        \
                                     order                 error order   
grid.n grid.Nx grid.Ny iter time                                         
3      17      17      241  3.11e-02   n/a              9.65e-04   n/a   
       34      34      514  1.17e-01  1.97              1.29e-04  2.90   
       68      68      1074 7.57e-01  1.98              1.68e-05  2.94   
       136     136     2243 7.75e+00  1.99              2.14e-06  2.97   

                                     $(\nabla\phi)^2_{CC}$        
                                                     error order  
grid.n grid.Nx grid.Ny iter time                                  
3      17      17      241  3.11e-02              5.26e-03   n/a  
       34      34      514  1.17e-01              1.33e-03  1.98  
       68      68      1074 7.57e-01              3.36e-04  1.98  
       136     136     2243 7.75e+00              8.45e-05  1.99

Direction is centered, n is 3


...

$\phi$       $(\nabla\phi)^2_{FW}$  \
                                         error order                 error   
grid.n grid.Nx grid.Ny iter time                                             
4      17      17      395  4.33e-02  4.36e-06   n/a              4.40e-05   
       34      34      850  3.08e-01  2.60e-07  4.07              2.97e-06   
       68      68      1776 2.47e+00  1.62e-08  4.01              1.93e-07   
       136     136     3711 2.55e+01  1.02e-09  3.99              1.23e-08   

                                           $(\nabla\phi)^2_{BW}$        \
                                     order                 error order   
grid.n grid.Nx grid.Ny iter time                                         
4      17      17      395  4.33e-02   n/a              3.12e-04   n/a   
       34      34      850  3.08e-01  3.89              3.72e-05  3.07   
       68      68      1776 2.47e+00  3.94              4.61e-06  3.01   
       136     136     3711 2.55e+01  3.97              5.77e-07  3.00   

                                     $(\nabla\phi)^2_{CC}$        
                                                     error order  
grid.n grid.Nx grid.Ny iter time                                  
4      17      17      395  4.33e-02              1.58e-04   n/a  
       34      34      850  3.08e-01              1.88e-05  3.07  
       68      68      1776 2.47e+00              2.32e-06  3.02  
       136     136     3711 2.55e+01              2.89e-07  3.00

Direction is backward, n is 4


$\phi$       $(\nabla\phi)^2_{FW}$  \
                                         error order                 error   
grid.n grid.Nx grid.Ny iter time                                             
4      17      17      395  4.31e-02  4.36e-06   n/a              3.12e-04   
       34      34      848  3.36e-01  2.60e-07  4.07              3.72e-05   
       68      68      1772 2.74e+00  1.62e-08  4.01              4.61e-06   
       136     136     3713 2.47e+01  1.02e-09  3.99              5.77e-07   

                                           $(\nabla\phi)^2_{BW}$        \
                                     order                 error order   
grid.n grid.Nx grid.Ny iter time                                         
4      17      17      395  4.31e-02   n/a              4.40e-05   n/a   
       34      34      848  3.36e-01  3.07              2.97e-06  3.89   
       68      68      1772 2.74e+00  3.01              1.93e-07  3.94   
       136     136     3713 2.47e+01  3.00              1.23e-08  3.97   

                                     $(\nabla\phi)^2_{CC}$        
                                                     error order  
grid.n grid.Nx grid.Ny iter time                                  
4      17      17      395  4.31e-02              1.58e-04   n/a  
       34      34      848  3.36e-01              1.88e-05  3.07  
       68      68      1772 2.74e+00              2.32e-06  3.02  
       136     136     3713 2.47e+01              2.89e-07  3.00

Direction is centered, n is 4


$\phi$       $(\nabla\phi)^2_{FW}$  \
                                         error order                 error   
grid.n grid.Nx grid.Ny iter time                                             
4      17      17      258  3.19e-02  8.80e-06   n/a              5.03e-04   
       34      34      561  2.28e-01  6.71e-07  3.71              7.64e-05   
       68      68      1190 1.94e+00  4.70e-08  3.84              1.07e-05   
       136     136     2509 1.92e+01  3.12e-09  3.91              1.43e-06   

                                           $(\nabla\phi)^2_{BW}$        \
                                     order                 error order   
grid.n grid.Nx grid.Ny iter time                                         
4      17      17      258  3.19e-02   n/a              5.03e-04   n/a   
       34      34      561  2.28e-01  2.72              7.64e-05  2.72   
       68      68      1190 1.94e+00  2.84              1.07e-05  2.84   
       136     136     2509 1.92e+01  2.91              1.43e-06  2.91   

                                     $(\nabla\phi)^2_{CC}$        
                                                     error order  
grid.n grid.Nx grid.Ny iter time                                  
4      17      17      258  3.19e-02              9.05e-05   n/a  
       34      34      561  2.28e-01              1.26e-05  2.84  
       68      68      1190 1.94e+00              1.72e-06  2.88  
       136     136     2509 1.92e+01              2.27e-07  2.92

We observe that the derivative, if computed with a different dG method, converges with one order less than the solution and the derivative with the correct dG method.
Further, we see that the centered derivative is order 3 for both n=3 and n=4 (!!)